Here is an implementation of a neural network for solving the problem of finding the optimal treatment size per person.


**problem:**

It is often too costly for companies to prescribe treatment randomly. In this regard, many classical approaches don't work correctly (_uplift trees, S/T/X learners etc_).
Person get the treatment according features.

**solution:**
This neural network is trying to learn how to cope with this non-randomness.
* First, the network is trained on all objects,
* then we devide batch into treatment and control groups,
* then we devide batch according to the magnitude of the treatment (here we'd like to find the effect of feature values on treatment assignment),
* then predict target and count loss.

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils import data
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16, 8)
import numpy as np
import pandas as pd

from IPython.display import clear_output

In [2]:
# set defaults
all_features = []
cat_features = []
cont_features = [x for x in all_features if x not in cat_features]
target_col = 'target'
treatment_col = 'treatment_size'
group_col = 'is_control_group'

In [19]:
learning_rate = 1e-3

epochs = 100
batch_size = 128
log_interval = 20

emb_cols = cat_features
n_cont = len(cont_feats)
emb_szs = [(None, 3), (None, 2), (None, 10)] # set according nunique values for each categorical feature
max_size = 30
treatments = [0, 10, 20, 30] # possible treatment sizes. (easily scalable to continuous case)

In [20]:
class OutcomeDataset(data.Dataset):
    def __init__(self, *, X, emb_cols, cont_feats):
        """

        :param X: DataFrame with all features
        :param emb_cols: Categorical features names
        :param cont_feats: Continuous features names
        """
        X = X.copy()
        self.ts = X['treatment_size'].values.astype(int)
        self.X1 = X[emb_cols].copy().values.astype(np.int64) #categorical columns
        self.X2 = X[cont_feats].copy().values.astype(np.float32) #numerical columns

        self.y = X['target'].values.astype(int)
        self.primary_key = X['primary_key'].values.astype(np.int64) # primary_key - object id

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.ts[idx], self.X1[idx], self.X2[idx], self.y[idx], self.primary_key[idx]

In [21]:
train_ds = OutcomeDataset(X=train, emb_cols=cat_feats, cont_feats=cont_feats)
valid_ds = OutcomeDataset(X=valid, emb_cols=cat_feats, cont_feats=cont_feats)

train_loader = data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

In [22]:
class MyNet(nn.Module):
    
    def __init__(self, *, 
                 output_dim: int, # First layer size
                 output_control_dim: int, # Control group layer size
                 output_treatment_dim: int,  # Treatment group layer size
                 output_size_dim: int,  #Treatment_size layer size
                 p: float, # probability for dropout
                 embedding_sizes: list,
                 n_cont: int,
                 treatments: list):
        super(MyNet, self).__init__()
        self.treatments = treatments
        self.drop_layer = nn.Dropout(p=p)
        self.emb_drop = nn.Dropout(0.6)
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes]) # эмбединги для категориальных фичей
        n_emb = sum(e.embedding_dim for e in self.embeddings) 
        self.n_emb, self.n_cont = n_emb, n_cont
        
        self.first_layer = nn.Linear(in_features=self.n_emb + self.n_cont, out_features=output_dim)
        self.bn = nn.BatchNorm1d(self.n_cont)
        
        self.control_layer = nn.Linear(in_features=output_dim, out_features=output_control_dim)
        self.treatment_layer = nn.Linear(in_features=output_dim, out_features=output_treatment_dim)
        
        d = dict.fromkeys([str(x) for x in self.treatments])
        for i in d:
            d[i]=nn.Linear(in_features=output_treatment_dim, out_features=output_size_dim)
        self.size_layers = nn.ModuleDict(d) # layers for treatment sizes
        
        
        self.output_layer = nn.Linear(in_features=output_size_dim, out_features=1)
        self.output_control_layer = nn.Linear(in_features=output_control_dim, out_features=1)
        
    def forward(self, x_cat, x_cont, y, ts, primary_key):
        """

        :param x_cat: categorical features
        :param: x_cont: numeric features
        :param y: target
        :param ts: treatment size
        :param primary_key: object id
        """
        output=[]
        output_y=[]
        output_ts = []
        output_primary_key = []

        x = [e(x_cat[:,i]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        
        x = self.first_layer(x)
        x = self.drop_layer(x)
        x = F.relu(x)
        
        if 0 in ts:
            x1 = self.control_layer(x[ts==0])
            x1 = F.relu(x1)
            x1 = self.output_control_layer(x1)
            x1 = torch.sigmoid(x1)
            output.append(x1)
            
            y_tmp = y[ts==0]
            output_y.append(y_tmp)
            ts_tmp = ts[ts==0]
            output_ts.append(ts_tmp)
            primary_key_tmp = primary_key[ts==0]
            output_primary_key.append(primary_key_tmp)
        
        x2 = self.treatment_layer(x[ts!=0])
        x2 = F.relu(x2)
        ts_n = ts[ts!=0]
        y_n = y[ts!=0]
        primary_key_n = primary_key[ts!=0]

        for s in self.treatments:
            if s in ts:
                h = self.size_layers[str(s)](x2[ts_n==s])
                h = self.drop_layer(h)
                h = F.relu(h)
                h = self.output_layer(h)
                h = torch.sigmoid(h)
                output.append(h)
                
                y_tmp = y_n[ts_n==s]
                output_y.append(y_tmp)
                ts_tmp = ts_n[ts_n==s]
                output_ts.append(ts_tmp)
                primary_key_tmp = primary_key_n[ts_n==s]
                output_primary_key.append(primary_key_tmp)
                
        return torch.cat((output)), (torch.cat((output_y))).type(torch.FloatTensor), output_ts, output_primary_key

In [23]:
net=MyNet(output_dim=128,
         output_control_dim=32,
         output_treatment_dim=32,
         output_size_dim = 16,
         p=0.3,
         embedding_sizes=emb_szs,
         n_cont=n_cont,
         treatments=treatments)
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min',
            factor=0.5, patience=10, threshold=0.001, verbose=True)
criterion = nn.BCELoss()

In [46]:
# train loop
best_score = None
history_train = []
history_valid = []

for epoch in range(epochs):
    slt=0
    slv=0
    net.train() # turn on train mode
    for batch_idx, (ts, x_cat, x_cont, target, primary_key) in enumerate(train_loader):
        x_cat, x_cont, target = Variable(x_cat), Variable(x_cont), Variable(target)
        optimizer.zero_grad()
        net_out, net_y, net_ts, net_primary_key = net(x_cat, x_cont, target, ts, primary_key)
        loss = criterion(net_out, net_y.unsqueeze(1))
        slt+=loss.item()
        loss.backward()
        optimizer.step()

    net.eval() # turn on eval mode
    with torch.no_grad():
        for batch_idx, (ts_val, x_cat_val, x_cont_val, target_val, primary_key_val) in enumerate(valid_loader):
            x_cat_val, x_cont_val, target_val = Variable(x_cat_val), Variable(x_cont_val), Variable(target_val)

            net_out_val, net_y_val, net_ts_val, net_primary_key_val = net(x_cat_val, x_cont_val, target_val, ts_val, primary_key_val)
            loss_valid = criterion(net_out_val, net_y_val.unsqueeze(1))
            slv+=loss_valid.item()
    scheduler.step(loss_valid)

    #  save current status if current valid loss is smaller than previous
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': net.cpu().state_dict(),
    'optimizer': optimizer.state_dict()
    }
    if best_score is None:
        best_score=loss_valid
    if loss_valid < best_score:
        torch.save(checkpoint, 'model_checkpoint.pt')

#     visualization
    history_train.append(slt/len(train_loader))
    history_valid.append(slv/len(valid_loader))

    if (epoch+1)%10 == 0:
        clear_output(True)
        print(f"Epoch: {epoch} | Mean loss train: {slt/len(train_loader)} | | Mean loss valid: {slv/len(valid_loader)}")
        plt.plot(history_train, 'b')
        plt.plot(history_valid, 'r')
        plt.legend(['train', 'valid'])
        plt.show();

net.eval()

MyNet(
  (drop_layer): Dropout(p=0.3, inplace=False)
  (emb_drop): Dropout(p=0.6, inplace=False)
  (embeddings): ModuleList(
    (0): Embedding(3, 3)
    (1): Embedding(2, 2)
    (2): Embedding(17, 10)
  )
  (first_layer): Linear(in_features=187, out_features=128, bias=True)
  (bn): BatchNorm1d(172, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (control_layer): Linear(in_features=128, out_features=32, bias=True)
  (treatment_layer): Linear(in_features=128, out_features=32, bias=True)
  (size_layers): ModuleDict(
    (1000): Linear(in_features=32, out_features=16, bias=True)
    (3000): Linear(in_features=32, out_features=16, bias=True)
    (500): Linear(in_features=32, out_features=16, bias=True)
    (5000): Linear(in_features=32, out_features=16, bias=True)
    (555): Linear(in_features=32, out_features=16, bias=True)
    (777): Linear(in_features=32, out_features=16, bias=True)
  )
  (output_layer): Linear(in_features=16, out_features=1, bias=True)
  (output_contr

In [47]:
def make_data(X=post):
    X = X.copy()
    ts = X['treatment_size'].values.astype(int)
    X1 = X[emb_cols].copy().values.astype(np.int64) #categorical columns
    X2 = X[cont_feats].copy().values.astype(np.float32) #numerical columns
    y = X['target'].values.astype(int)
    primary_key = X['primary_key'].values.astype(np.int64)
    return ts, Variable(torch.Tensor(X1).long()), Variable(torch.Tensor(X2)), Variable(torch.Tensor(y)), primary_key
def make_ts(value=0, size=0):
    return np.array([value]*size)

In [57]:
def make_stats(X=post):
    ts, X1, X2, y, primary_key = make_data(X=X)
    output={}
    size = ts.shape[0]
    for i in treatments:
        print(i)
        tsto = make_ts(value=i, size=size)
        outputs, out_y, out_ts, out_primary_key = net(X1, X2, y, tsto, primary_key)
        output[i]=[x.item() for  x in outputs]
    return primary_key, ts, pd.DataFrame(output), y

In [ ]:
output_primary_key, ts_out, output_post, y_out = make_stats(post)

In [59]:
res = pd.concat([pd.DataFrame({'primary_key': output_primary_key,
                                'treatment_size': ts_out,
                                'target': y_out}),
                 output_post], axis=1)